## Fixing/Modifying individual file (EIGENSTRAT)

### Fixing populations after convertf

Individual file (commonly *.ind) contains population information for samples.
Here: collect 4 different Metadata files, as pandas DFs, combine them and use as lookup, when fixing the original *.ind file that did not have that info after convertf.

### ind file variations
The 2. purpose is to explore population definitions and accordingly write
Also, writing separate popfiles (as used by qp3Pop) for Northern and Southern Dispersal Route. Finally, creating ind file for use popfile


In [1]:
from collections import Counter

import pandas as pd

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))


/tmp/ipykernel_772574/731169535.py:5: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


In [3]:
import re
def readZallouaMetadata(manifest, selcols, colnames=['Id','Region']):    
    regionalContext = pd.read_csv(manifest, skiprows=8, sep=',', skip_blank_lines=True)
    regionalContext = regionalContext.iloc[:,selcols]
    regionalContext.columns = colnames
    regionalContext.set_index('Id', inplace=True)
    return regionalContext.dropna(subset=colnames[1:])
yemenMeta = readZallouaMetadata("../Metadata/3577stdy_manifest_3450_190315.csv.gz", [2,3], colnames=['Id','RegionId'])
yemenMeta["Region"] = [re.findall('\D+', sup)[0] for sup in yemenMeta['RegionId']]
yemenMeta = yemenMeta[['Region']]
yemenMeta = yemenMeta[yemenMeta.Region!='Empty']
chadContext = readZallouaMetadata("../Metadata/yemcha_manifest_3558_160415.csv.gz", [2,8])
pakistan = readZallouaMetadata("../Metadata/sc_egyptlc_manifest_1764_050613.csv.gz", [2,8])
pakistan.head()

,Region
Id,
SC_EGYPTLC5544382,Pakistan
SC_EGYPTLC5544383,Pakistan
SC_EGYPTLC5544384,Pakistan
SC_EGYPTLC5544385,Pakistan
SC_EGYPTLC5544386,Pakistan


In [7]:
manifest, selcols, colnames= "../Metadata/3577stdy_manifest_3450_190315.csv.gz", [2,3], ['Id','RegionId']
t = pd.read_csv(manifest, skiprows=8, sep=',', skip_blank_lines=True)
t = t.iloc[:,selcols]
t
#regionalContext.columns = colnames

,SANGER SAMPLE ID,SUPPLIER SAMPLE NAME
0,3577STDY6068360,Dal1
1,3577STDY6068361,Ibb15
2,3577STDY6068362,Rsa18
3,3577STDY6068363,Rsa35
4,3577STDY6068364,Rsa47
...,...,...
9985,NaN,NaN
9986,NaN,NaN
9987,NaN,NaN
9988,NaN,NaN


In [4]:
yemenMeta.head()

,Region
Id,
3577STDY6068360,Dal
3577STDY6068361,Ibb
3577STDY6068362,Rsa
3577STDY6068363,Rsa
3577STDY6068364,Rsa


In [11]:
reichMeta=pd.read_csv("Reich/v44.3_HO_public.anno", sep='\t')
reichMeta.columns = ['Index', 'Id', 'Id2',
       'Publication', 'contact',
       'Date',
       'Full_Date',
       'Group_Label', 'Locality', 'Region', 'Lat', 'Long',
       'Data source', 'Cov_autosm',
       'SNPs_autosm', 'Sex',
       'Library_type',
       'ASSESSMENT']

reichMetaNow = reichMeta[reichMeta.Date==0]
reichMetaAncient = reichMeta[reichMeta.Date>1000]
cat0 = ["Ancient", "Contemporary"]
cat1 = ["Population (Group_Label)", "Country/Region"]
for i0, df in enumerate([reichMetaAncient, reichMetaNow]):
    c1 = Counter(df.Group_Label)
    #c2 = Counter(df.Locality)
    c3 = Counter(df.Region)

    for i1, c in enumerate([c1,c3]):
        print(cat0[i0], cat1[i1])
        print(c.most_common(50), len(c))
        print('-' * 50)
    
reichMeta[['Id', 'Group_Label', 'Locality', 'Region']]


Ancient Population (Group_Label)
[('Spain_C', 77), ('Germany_EN_LBK', 54), ('France_MN', 43), ('Spain_MLN', 42), ('Switzerland_LN', 39), ('Germany_BellBeaker', 37), ('Czech_BellBeaker', 36), ('Russia_MLBA_Sintashta', 35), ('Italy_Imperial.SG', 35), ('Spain_EBA', 34), ('Taiwan_Hanben_IA', 34), ('Estonia_EarlyViking.SG', 34), ('Israel_MLBA', 32), ('Scotland_N', 30), ('Germany_Lech_EBA', 30), ('Serbia_IronGates_Mesolithic', 29), ('Pakistan_Loebanr_IA', 29), ('Denmark_Viking.SG', 29), ('Ireland_MN.SG', 26), ('England_C_EBA', 25), ('Turkey_Alalakh_MLBA', 25), ('India_RoopkundA', 23), ('Russia_Afanasievo', 22), ('Pakistan_Katelai_IA', 22), ('England_N', 21), ('Germany_EarlyMedieval.SG', 21), ('Turkmenistan_C_Geoksyur', 20), ('Mongolia_LBA_Khovsgol_6', 20), ('Turkmenistan_Gonur_BA_1', 19), ('England_BellBeaker', 19), ('Cuba_CanimarAbajo_Archaic', 19), ('Italy_Sardinia_EBA', 18), ('England_Viking.SG', 18), ('Jordan_LBA', 17), ('England_N.SG', 17), ('Germany_Lech_EBA_lc', 17), ('Turkey_Arslante

,Id,Group_Label,Locality,Region
0,MAL-005,Malawi_Yao,Dedza // Yao,Malawi
1,MAL-009,Malawi_Yao,Machinga // Yao,Malawi
2,MAL-011,Malawi_Chewa,Mchinga // Chichewa,Malawi
3,MAL-012,Malawi_Chewa,Salima // Chichewa,Malawi
4,MAL-014,Malawi_Chewa,Nambuma // Chichewa,Malawi
...,...,...,...,...
13192,VK94.SG,Denmark_Viking.SG,"Sealand, Gl. Lejre",Denmark
13193,VK95.SG,Iceland_Viking.SG,Hofstadir,Iceland
13194,VK98.SG,Iceland_Viking.SG,Hofstadir,Iceland
13195,VK99.SG,Iceland_Viking.SG,Hofstadir,Iceland


### Combining and saving metadata

In [12]:
## Using Group_Label (population name) as "Region" label (for compatibility)
reichMeta = reichMeta[['Id', 'Group_Label']].rename(columns={'Group_Label':"Region"}).set_index('Id')

meta = pd.concat([yemenMeta, chadContext, pakistan, reichMeta])
meta.to_csv("Metadata/allmetaRegions.csv")

## AdmixTools - F3 stats
Using qp3Pop to calculate F3 statistics
### Prepping the input parameter files
The parameter file is the results of the merged plink file (Yemen, UAE, context), converted to EIGENSTRAT/AdmixTool format (geno).
The population file is what lists all populations. Starting with a little QC:
* exclude populations denoted with an 'Ignore' prefix
* only keep populations that have at least 10 samples


In [13]:
Counter(meta.Region)

Counter({'Dal': 9,
         'Ibb': 25,
         'Rsa': 48,
         'Dhm': 10,
         'Abyn': 10,
         'Sad': 10,
         'Mhw': 10,
         'Byd': 7,
         'Lahj': 10,
         'Jwf': 10,
         'San': 10,
         'Hdr': 25,
         'Mrb': 10,
         'Amr': 10,
         'Haj': 10,
         'Shb': 10,
         'Tiz': 24,
         'Chad': 276,
         'Pakistan': 48,
         'Malawi_Yao': 9,
         'Malawi_Chewa': 11,
         'Malawi_Ngoni': 4,
         'Malawi_Tumbuka': 10,
         'Karachai': 12,
         'Adygei': 31,
         'Altaian_Chelkan': 8,
         'Tubalar': 27,
         'Ignore_Tubalar(PCA_outlier)': 1,
         'Altaian': 24,
         'Ignore_Altaian(PCA_outlier)': 1,
         'Nivh': 10,
         'Evenk_FarEast': 5,
         'Negidal': 3,
         'Nanai': 10,
         'Ignore_Negidal(PCA_outlier)': 1,
         'Armenian_Hemsheni': 8,
         'Azeri': 17,
         'Bashkir': 53,
         'Buryat': 37,
         'Khamnegan': 10,
         'Ignore_Bur

In [211]:
## all (other) pops
opops = [pop for pop, count in Counter(meta.Region).most_common() if (not pop.startswith('Ignore') and count>=10)]

In [214]:
## What is the closest population to UAE_ME/UAE_CSA/Hdr/Rsa/Ibb/Tiz/Sad

parfileText = """genotypename:   swAsia_uae_reichHO_final_QC.geno
snpname:        swAsia_uae_reichHO_final_QC.snp
indivname:      swAsia_uae_reichHO_final_QC4.ind
popfilename:    %s
inbreed:	YES"""

def prepParameterFiles(pop, otherPops, outgrp="Mbuti", popfile='popfile_%s_vAll.txt', parameterFile = 'parfile_qp3Pop_%s', dry=False)
    popfilename = popfile % pop
    parfilename = paramterFile % pop

    with open(popfilename, 'w') as popfile:
        for pop2 in opops:
            if pop != pop2 and pop!= outgrp and pop2 != outgrp:
                print(pop, pop2, outgrp, file=popfile)
    with open(parfilename, 'w') as parfile:
        print(parfileText % popfilename, file=parfile)
    cmd = f"qp3Pop -p {parfilename} > qp3Pop_{pop}.log &"
    print(cmd)

arabpops = 'Hdr/Rsa/Ibb/Tiz/Sad'.split('/')
for pop in uaepops:
    


qp3Pop -p parfile_qp3Pop_UAE_ME > qp3Pop_UAE_ME.log &
qp3Pop -p parfile_qp3Pop_UAE_CSA > qp3Pop_UAE_CSA.log &
qp3Pop -p parfile_qp3Pop_Hdr > qp3Pop_Hdr.log &
qp3Pop -p parfile_qp3Pop_Rsa > qp3Pop_Rsa.log &
qp3Pop -p parfile_qp3Pop_Ibb > qp3Pop_Ibb.log &
qp3Pop -p parfile_qp3Pop_Tiz > qp3Pop_Tiz.log &
qp3Pop -p parfile_qp3Pop_Sad > qp3Pop_Sad.log &


In [15]:
def test(pop='uae', fn="hallo_%s_world"):
    print(fn % pop)
test(fn='hello_%s_welt')

hello_uae_welt


In [207]:
allRegions

,Region
Id,
3577STDY6068360,Dal
3577STDY6068361,Ibb
3577STDY6068362,Rsa
3577STDY6068363,Rsa
3577STDY6068364,Rsa
...,...
120421,UAE_Misc
10411,UAE_Misc
10449,UAE_ME


In [209]:

with open('swAsia_uae_reichHO_final_QC4.ind', 'w') as ind:
    for line in open('swAsia_uae_reichHO_final_QC.ind_bak'):
        fields=line.split()
        f = fields[0]
        if f.startswith('urn:wtsi'):
            f = '_'.join(f.split('_')[2:])        
        region = allRegions['Region'].get(f, 'Control')
        print(line[:line.index(fields[2])] + region, file=ind)


In [158]:

pops = {'levantPops': 'Egyptian Druze Druze.SDG Palestinian BedouinB BedoiunA.SDG Iranian Jordanian'.split(),
        'arabPops': 'UAE_ME Yemeni_Highlands Yemeni_Desert2 Yemeni_Northwest Rsa Ibb Hdr Tiz Dhm Abyn Sad Mhw Lahj Jwf San Mrb Amr Haj Shb Somali.DG1 Eritrea Saudi'.split()
       }

B = 'Chad' # 'Dinka' 'Shaigi.WGA'
outgroup = 'Mbuti' # has few members only

for popgroup, pops in pops.items():
    with open(f'swAsia_uae_reichHO_final_QC_{popgroup[:-4]}_{B}_{outgroup}.pop', 'w') as popfile:        
        for pop in pops:
            print(pop, B, outgroup, file=popfile)
        

In [150]:
ls -lt |head

total 5413508
-rw-rw-r--. 1 ahenschel ahenschel        425 Oct 22 17:42 swAsia_uae_reichHO_final_QC_arab.pop
-rw-rw-r--. 1 ahenschel ahenschel        134 Oct 22 17:42 swAsia_uae_reichHO_final_QC_levant.pop
-rw-rw-r--. 1 ahenschel ahenschel     151279 Oct 22 17:42 fixIndivPopulation.ipynb
-rw-rw-r--. 1 ahenschel ahenschel       5599 Oct 22 16:55 jupyter3.log
-rw-rw-r--. 1 ahenschel ahenschel     611192 Oct 22 16:55 metadataExplorer.ipynb
-rw-rw-r--. 1 ahenschel ahenschel     588927 Oct 22 16:07 swAsia_uae_reichHO_final_QC3.ind
drwxrwxr-x. 1 ahenschel ahenschel        980 Oct 22 12:43 UAE/
-rw-rw-r--. 1 ahenschel ahenschel    2898205 Oct 22 11:04 zalloua480_reich_PCA.ipynb
-rw-rw-r--. 1 ahenschel ahenschel          0 Oct 22 10:51 jupyter4.log
ls: write error


In [151]:
!more swAsia_uae_reichHO_final_QC_levant.pop

Druze Chad Yoruba
Palestinian Chad Yoruba
BedouinB.SDG Chad Yoruba
BedoiunA.SDG Chad Yoruba
Iranian Chad Yoruba
Jordanian Chad Yoruba


In [59]:
%pprint
len(sorted(chadContext.index))

Pretty printing has been turned ON


276

In [54]:
chadContext.index

,Region
Id,
yemcha6089628,Chad
yemcha6089629,Chad
yemcha6089630,Chad
yemcha6089631,Chad
yemcha6089632,Chad
...,...
yemcha6089909,Chad
yemcha6089910,Chad
yemcha6089912,Chad


In [4]:
from collections import Counter
Counter('_'.join(line.split()[2:]) for line in open('swAsia_uae_reichHO_final_QC.ind')).most_common()


[('Russia', 1705),
 ('China', 1138),
 ('UAE', 996),
 ('Spain', 680),
 ('Pakistan', 657),
 ('Italy', 538),
 ('USA', 446),
 ('Great_Britain', 412),
 ('Israel', 369),
 ('Germany', 356),
 ('Nigeria', 352),
 ('France', 286),
 ('India', 254),
 ('Chad', 245),
 ('Turkey', 241),
 ('Kenya', 220),
 ('Hungary', 215),
 ('Sweden', 193),
 ('Japan', 176),
 ('Peru', 170),
 ('Kazakhstan', 166),
 ('Vietnam', 148),
 ('Mongolia', 139),
 ('Puerto_Rico', 130),
 ('Iran', 128),
 ('Dominican_Republic', 127),
 ('Mexico', 123),
 ('Finland', 122),
 ('Gambia', 121),
 ('Uzbekistan', 113),
 ('Poland', 110),
 ('Colombia', 108),
 ('Yemen', 107),
 ('Sri_Lanka', 103),
 ('Ukraine', 98),
 ('Estonia', 96),
 ('Bangladesh', 96),
 ('Barbados', 96),
 ('Sierra_Leone', 95),
 ('Denmark', 95),
 ('Turkmenistan', 92),
 ('Czech_Republic', 87),
 ('Switzerland', 80),
 ('Papua_New_Guinea', 76),
 ('Brazil', 72),
 ('Greece', 69),
 ('Taiwan', 69),
 ('Algeria', 66),
 ('Nepal', 66),
 ('Lebanon', 65),
 ('Ireland', 63),
 ('Bulgaria', 58),
 ('Ic

In [57]:
df=pd.DataFrame({'Exp':[25,25,25,25], 'Obs':[20,25,20,35]})

In [56]:
import scipy

In [58]:
from scipy.stats import chisquare
chisquare(df.Obs, df.Exp)

Power_divergenceResult(statistic=6.0, pvalue=0.11161022509471268)